In [ ]:
import parse
from collections import deque
from copy import deepcopy
import itertools

def deck(n):
    return deque(range(n))

def part_1(shuffle, cards):
    # I'd prefer to use pyrsistent but it's incredibly slow here
    cards = deepcopy(cards)
    n = len(cards)

    for line in shuffle:
        line = line.strip()
        
        if line == 'deal into new stack':
            cards.reverse()
        elif r := parse.parse('cut {cut:d}', line):
            cards.rotate(-r['cut'])
        elif r := parse.parse('deal with increment {incr:d}', line):
            new_cards = [-1] * n
            for i in range(n):
                new_cards[(i * r['incr']) % n] = cards[i]
            cards = deque(new_cards)
        else:
            print('Unparsed line', line)
            
    return cards

In [2]:
test_deck = deck(10)

part_1(['deal into new stack'], test_deck)

deque([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [3]:
part_1(['cut 3'], test_deck)

deque([3, 4, 5, 6, 7, 8, 9, 0, 1, 2])

In [4]:
part_1(['cut -4'], test_deck)

deque([6, 7, 8, 9, 0, 1, 2, 3, 4, 5])

In [5]:
part_1(['deal with increment 3'], test_deck)

deque([0, 7, 4, 1, 8, 5, 2, 9, 6, 3])

In [6]:
test_shuffle = '''deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1'''.splitlines()

part_1(test_shuffle, test_deck)

deque([9, 2, 5, 8, 1, 4, 7, 0, 3, 6])

In [7]:
shuffle = open('input').read().strip().splitlines()
full_deck = deck(10007)

In [8]:
part_1(shuffle, full_deck)[2019]

9093

Wrong. Hmm.

OFFFS. After a large amount of fruitless debugging it turns out I was answering the wrong question.

In [9]:
part_1(shuffle, full_deck).index(2019)

4649

## Part 2

Right, I'm going to have to do some modular arithmetic. Is the size of the deck prime?

In [10]:
deck_size = 119315717514047

In [11]:
import math

math.sqrt(deck_size)

10923173.417741155

In [12]:
[n for n in range(2, int(_)) if n % deck_size == 0]

[]

Yes it is.

Rewrite part 1 as a function $ptr = (ax+b) \mod{m}$, where $m$ is the size of the deck, $x$ the original position, and $ptr$ the landing position.

In [13]:
def find_a_b(shuffle, deck_length):
    a = 1
    b = 0
    
    for line in shuffle:
        line = line.strip()

        if line == 'deal into new stack':
            a = -a
            b = -b - 1
        elif r := parse.parse('cut {cut:d}', line):
            b = b - r['cut']
        elif r := parse.parse('deal with increment {incr:d}', line):
            a = a * r['incr']
            b = b * r['incr']
        else:
            print('Unparsed line', line)

    return (a % deck_length, b % deck_length)


def part_1_v2(shuffle, deck_length):
    a, b = find_a_b(shuffle, deck_length)
    
    shuffled = [-1] * deck_length
    for x in range(deck_length):
        shuffled[(a * x + b) % deck_length] = x
        
    return shuffled

In [14]:
part_1_v2(['deal into new stack'], 10)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [15]:
part_1_v2(['cut 3'], 10)

[3, 4, 5, 6, 7, 8, 9, 0, 1, 2]

In [16]:
part_1_v2(['cut -4'], 10)

[6, 7, 8, 9, 0, 1, 2, 3, 4, 5]

In [17]:
part_1_v2(['deal with increment 3'], 10)

[0, 7, 4, 1, 8, 5, 2, 9, 6, 3]

In [18]:
part_1_v2(test_shuffle, 10)

[9, 2, 5, 8, 1, 4, 7, 0, 3, 6]

In [19]:
part_1_v2(shuffle, 10007).index(2019)

4649

In [20]:
p = 10007
a, b = find_a_b(shuffle, p)

In [21]:
a, b

(9520, 7216)

But we need to find the inverse of this $x = (ptr - b)/a \mod m$. The inverse of $a$ modulo $p$ is

In [22]:
a_inv = pow(a, -1, 10007)
a_inv

4685

In [23]:
a_inv * a % p

1

In [24]:
(4649 - b) * a_inv % p

2019

That looks good.

But we need to apply this $n=101741582076661$ times, i.e.

$ax + b$


$a(ax + b) + b = a^2x + ab + b$

$a(a^2x + ab + b) + b = a^3x + a^2b + ab + b$

etc, which are geometric progressions if we ignore the $x$ factor. So

$a^nx + a^{n-1}b + a^{n-2}b + \ldots$

becomes

$a^nx + b(1-a^n)/(1-a)$

In [25]:
n = 101741582076661
p = 119315717514047
a, b = find_a_b(shuffle, p)
a, b

(6871724559536, 20103085772277)

We then get a new $a$

In [26]:
multi_a = a^n
multi_a

99744112723525

and $b$

In [27]:
multi_b = b * (1 - a^n) * pow(a, -1, p) % p
multi_b

84191803482951

In [28]:
multi_a_inv = pow(a, -1, p)
(2020 - multi_b) * multi_a_inv % p

17373296509610

Wrong.